In [27]:
!pip install pandas openpyxl xlrd


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import json
def json_to_dict_and_dataframe(file_path):
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f) 
    dict_data = {}
    for item in data:
        dict_data[item["name"]] = item # item = 'station_id': 74,'name': "Sotz'il - Waqxaqi'B'alam", 'latitude': 14.67, 'longitude': -90.81, 'altitude': 1781,'stratum': 'Alto'

    return dict_data

In [29]:
import pandas as pd

file_path = './data/xls_cleaned/20241003-Reporte.xls'
df = pd.read_excel(file_path, engine='xlrd')  
print(df.columns)

Index(['Estacion', 'Fecha', 'radiacion', 'humedad relativa', 'precipitacion',
       'temperatura', 'velocidad viento', 'mojadura', 'direccion viento',
       'eto', 'indice calor'],
      dtype='object')


In [30]:
import datetime

def extract_xls_into_dataframe(file_path, stations_dict):
    df = pd.read_excel(file_path, engine='xlrd')
    df = df.drop(columns=["eto"])
    df = df.rename(columns={
        'Estacion': 'name', 
        "Fecha": "date_time", 
        "radiacion": "radiation",
        "humedad relativa": "relative_humidity",
        "precipitacion": "precipitation",
        "temperatura": "temperature",
        "velocidad viento": "wind_speed",
        "mojadura": "wetness",
        "direccion viento": "wind_direction",
        "indice calor": "heat_index"        
    })
    
    df['station_id'] = df['name'].map(lambda x: stations_dict.get(x, {}).get('station_id'))
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M')
    df['date_time'] = df['date_time'].dt.tz_localize('America/Guatemala')

    df = df.drop(columns=['name'])
    
    return df    


In [31]:
example_file_path = './data/xls_cleaned/20241003-Reporte.xls'
stations_dict = json_to_dict_and_dataframe('./data/stations.json')
dataframe = extract_xls_into_dataframe(example_file_path, stations_dict)
dataframe.head()

,date_time,radiation,relative_humidity,precipitation,temperature,wind_speed,wetness,wind_direction,heat_index,station_id
0,2024-10-03 00:00,0.0,100,0.0,24.3,0.0,10,104.2,23.38,45
1,2024-10-03 00:15,0.0,100,0.0,24.3,0.0,10,25.6,23.38,45
2,2024-10-03 00:30,0.0,100,0.0,24.3,0.0,10,29.4,23.38,45
3,2024-10-03 00:45,0.0,100,0.0,24.2,0.0,10,25.3,23.12,45
4,2024-10-03 01:00,0.0,100,0.0,24.0,0.0,10,48.5,22.60,45


In [ ]:

from postgresql import get_connection

from dotenv import dotenv_values
envs = dotenv_values(".env")

async def upload_into_register_table(dataframe):
    conn = await get_connection(
        user=envs["PG_USER"],
        password=envs["PG_PASSWORD"],
        database=envs["PG_DATABASE"],
        host=envs["PG_HOST"]
    )
    query = """
    INSERT INTO StationRegisters (
        station_id, date_time, temperature, radiation, 
        relative_humidity, precipitation, wind_speed, 
        wetness, wind_direction, heat_index
    ) VALUES ($1, $2, $3, $4, $5, $6, $7, $8, $9, $10)
    ON CONFLICT (station_id, date_time) DO NOTHING
    """
    for _, row in dataframe.iterrows():
        await conn.execute(
            query,
            row['station_id'], row['date_time'], row['temperature'], 
            row['radiation'], row['relative_humidity'], row['precipitation'], 
            row['wind_speed'], row['wetness'], row['wind_direction'], row['heat_index']
        )

In [32]:
async def upload_xls_file_to_db_pipeline(file_name, dir_path):
    file_path = os.path.join(dir_path, file_name)
    stations_dict = json_to_dict_and_dataframe('./data/stations.json')
    dataframe = extract_xls_into_dataframe(file_path, stations_dict)
    await upload_into_register_table(dataframe)

    return

In [ ]:
# Executed on script since jupyter won't run async functions. 
import os
import asyncio

directory_path = './data/'
files_to_load = os.listdir("./data/xls_cleaned/") 
async def upload_multiple_files(): 
    for file_name in files_to_load:
        await upload_xls_file_to_db_pipeline(file_name, directory_path)
        
        
if __name__ == "__main__":
    asyncio.run(upload_multiple_files())